In [12]:
import pandas as pd
import os
import numpy as np

In [ ]:
# WORKING_DIR = os.path.dirname(os.path.abspath(__file__))

In [3]:
#reading user file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(filepath_or_buffer='E:\\taolao\\a.NgocBui\\recommend\\data\\ml-100k\\u.user', sep='|', names=u_cols, encoding='latin_1')
n_users = users.shape[0]
print('Number of users: ', n_users)

Number of users:  943


In [4]:
print(users.head())

user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


In [5]:
#reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings_base = pd.read_csv(filepath_or_buffer='E:\\taolao\\a.NgocBui\\recommend\\data\\ml-100k\\ua.base', sep='\t', names=r_cols, encoding='latin_1')
# print(ratings_base.head())
ratings_test = pd.read_csv(filepath_or_buffer='E:\\taolao\\a.NgocBui\\recommend\\data\\ml-100k\\ua.test', sep='\t', names=r_cols, encoding='latin_1')

rate_train = ratings_base.values
rate_test = ratings_test.values

print('number of training rates: ', rate_train.shape[0])
print('number of rating test: ', rate_test.shape[0])

number of training rates:  90570
number of rating test:  9430


In [6]:
#reading items file:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(filepath_or_buffer='E:\\taolao\\a.NgocBui\\recommend\\data\\ml-100k\\u.item', sep='|', names=i_cols, encoding='latin_1')
n_items = items.shape[0]
print('number of items: ', n_items)
print(items.head())

number of items:  1682
   movie id        movie title release date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children's  ...  Fantasy  Film-Noir  Horror  Musical  \
0          0          1           1  ...        0     

In [25]:
#Vì ta đang dựa trên thể loại của phim để xây dựng profile, ta sẽ chỉ quan tâm tới 19 giá trị nhị phân ở cuối mỗi hàng:
X0 = items.values
X_train_counts = X0[:, -19:]
print(X_train_counts.shape)

(1682, 19)


In [10]:
#tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=True, norm='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
# mỗi hàng của tfidf tương ứng với feature vector của 1 bộ phim

In [11]:
print(tfidf)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.53676706 0.65097024 ... 0.53676706 0.         0.        ]
 [0.         0.         0.         ... 1.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [15]:
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores),            time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:, 0] # all users
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id + 1)[0]
    item_ids = rate_matrix[ids, 1] - 1 # index starts from 0
    scores = rate_matrix[ids, 2]
    return (item_ids, scores)

In [16]:
from sklearn.linear_model import Ridge
from sklearn import linear_model

d = tfidf.shape[1] #data dim
W = np.zeros((d, n_users))
b = np.zeros((1, n_users))

for n in range(n_users):
    ids, scores = get_items_rated_by_user(rate_train, n)
    clf = Ridge(alpha=0.01, fit_intercept=True)
    Xhat = tfidf[ids, :]

    clf.fit(Xhat, scores)
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

In [20]:
Yhat = tfidf.dot(W) + b

In [24]:
# user owns id = 10
n = 10
np.set_printoptions(precision=2) # 2 digis after
ids, scores = get_items_rated_by_user(rate_test, n)
# print(ids)
# print(scores)
Yhat[n, ids]
print('Rated movies ids :', ids )
print('True ratings     :', scores)
print('Predicted ratings:', Yhat[ids, n])

Rated movies ids : [ 37 109 110 226 424 557 722 724 731 739]
True ratings     : [3 3 4 3 4 3 5 3 3 4]
Predicted ratings: [3.18 3.13 3.42 3.09 3.35 5.2  4.01 3.35 3.42 3.72]


In [31]:
def evaluate(Yhat, rates, W, b):
    se = 0
    cnt = 0
    for i in range(n_users):
        ids, scores_truth = get_items_rated_by_user(rates, n)
        scores_pred = Yhat[ids, n]
        e = scores_truth - scores_pred
        se += (e*e).sum(axis=0)
        cnt += e.size
    return np.sqrt(se/cnt)
print ('RMSE for training:', evaluate(Yhat, rate_train, W, b))
print ('RMSE for test    :', evaluate(Yhat, rate_test, W, b))

RMSE for training: 0.8676014628392172
RMSE for test    : 0.8386423938638295
